In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,AdaBoostClassifier
from xgboost import XGBClassifier

from sklearn import metrics

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# load data
train_df = pd.read_csv('/kaggle/input/forest-cover-type-prediction/train.csv')
test_df = pd.read_csv('/kaggle/input/forest-cover-type-prediction/test.csv')

or_test_df = test_df

In [ ]:
# overlook of data
train_df.head(n=2)

In [ ]:
# Check for Nans
train_df.isnull().values.any()

In [ ]:
train_df.columns

In [ ]:
train_df = train_df.drop(['Id'], axis=1)
test_df = test_df.drop(['Id'], axis=1)

In [ ]:
value_columns = ['Elevation', 'Aspect', 'Slope',
       'Horizontal_Distance_To_Hydrology', 'Vertical_Distance_To_Hydrology',
       'Horizontal_Distance_To_Roadways', 'Hillshade_9am', 'Hillshade_Noon',
       'Hillshade_3pm', 'Horizontal_Distance_To_Fire_Points']

In [ ]:
# Feature Engineering
# train_df['Hillshade'] = train_df['Hillshade_9am'] + train_df['Hillshade_Noon'] + train_df['Hillshade_3pm']

In [ ]:
def plot_features(features):
    for feature in features:
        plt.rcParams["figure.figsize"] = (10,3)
        for ctype in list(train_df['Cover_Type'].unique()):
            values = train_df.loc[train_df['Cover_Type'] == ctype][feature].values
            plt.scatter(x=values, y=np.arange(values.size), label=f"Cover type {ctype}")
        plt.title(feature)
        plt.legend()
        plt.show()

In [ ]:
# Scatter plots for features with contnous values
plot_features(value_columns)

**Elevation, Horizontal_Distance_To_Roadways and Horizontal_Distance_To_Fire_Points seem to be good features to preprocess. Wilderness_Area and Soil_Types themselves act as one hot encoded features**

In [ ]:
scaler = StandardScaler()
scaler.fit(train_df[value_columns])

In [ ]:
train_df[value_columns] = scaler.transform(train_df[value_columns])
test_df[value_columns] = scaler.transform(test_df[value_columns])

In [ ]:
train_labels = train_df['Cover_Type']
train_input = train_df.drop(['Cover_Type'], axis=1)

test_input = test_df[train_input.columns]

In [ ]:
model = XGBClassifier(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)
model.fit(train_input, train_labels)

In [ ]:
test_input.shape

In [ ]:
predictions=model.predict(test_input)

In [ ]:
results = pd.DataFrame({'Id': or_test_df['Id'], 'Cover_Type': predictions})

In [ ]:
results.head()

In [ ]:
results.to_csv("submissions.csv", index=False)